## RECT

In [ ]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
from tests.domain.primary.helpers.DomainTables import DomainTables

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float64'
dim=2
mesh_name = 'rectangles.msh'

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'helpers', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=7, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

In [ ]:
import importlib

module = importlib.import_module("helpers.TablesTestRect2D")
importlib.reload(module)
TestTablesRect2D = getattr(module, "TablesTestRect2D")

module = importlib.import_module("helpers.Checker2D")
importlib.reload(module)
Checker2D = getattr(module, "Checker2D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TestTablesRect2D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = Checker2D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_face_info()
checker.test_node_info()
checker.test_halo_info()
checker.summary()


## Triangles

In [ ]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
from tests.domain.primary.helpers.DomainTables import DomainTables

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float64'
dim=2
mesh_name = 'triangles.msh'

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'helpers', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=7, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

In [ ]:
import importlib

module = importlib.import_module("helpers.TablesTestTriangles2D")
importlib.reload(module)
TestTablesTriangles2D = getattr(module, "TablesTestTriangles2D")

module = importlib.import_module("helpers.Checker2D")
importlib.reload(module)
Checker2D = getattr(module, "Checker2D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TestTablesTriangles2D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = Checker2D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_face_info()
checker.test_node_info()
checker.test_halo_info()
checker.summary()

## Gamma in different domain

In [ ]:
domain_tables_7 = DomainTables(nb_partitions=7, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
domain_tables_5 = DomainTables(nb_partitions=5, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

def glob_to_loc(domain):
  loctoglob = domain.d_cell_loctoglob
  dic = {}
  for p in range(len(loctoglob)):
    for i in range(len(loctoglob[p])):
      g_index = loctoglob[p][i]
      dic[g_index] = (p, i)
  return dic

dic7 = glob_to_loc(domain_tables)
dic5 = glob_to_loc(domain_tables_5)

def get_cell_faces_vertices(partitionId, cellId, domain):
  c_faces = domain.d_cell_faces[partitionId][cellId]
  c_faces = c_faces[0:c_faces[-1]]

  c_faces_nodes = domain.d_faces[partitionId][c_faces]
  c_faces_nodes = c_faces_nodes[:, 0:2]

  node_vertices = domain.d_nodes[partitionId][c_faces_nodes][:, :, 0:2]
  return node_vertices

a = get_cell_faces_vertices(partitionId=5, cellId=0, domain=domain_tables) # domain split into 7 part
b = get_cell_faces_vertices(partitionId=3, cellId=9, domain=domain_tables_5) # domain split into 5 part
# (5, 0) and (3, 9) refer to the same cell
np.testing.assert_array_almost_equal(a, b) # they have different face nodes arrangement
# cell_7 = [[p1, p2], [p1, p2], [p1, p2]]
# cell_5 = [[p2, p1], [p1, p2], [p1, p2]]

for i in range(200):
  print(i, dic7[i], dic5[i])
  a = get_cell_faces_vertices(partitionId=5, cellId=0, domain=domain_tables)
  b = get_cell_faces_vertices(partitionId=3, cellId=9, domain=domain_tables_5)
  np.testing.assert_array_almost_equal(a, b)